# pneumonia mnist resnet18 (colab ready)

simple end to end notebook for training and explaining a resnet18 on pneumonia mnist.


## setup

* switch runtime to gpu in colab (runtime > change runtime type > gpu)
* mount drive so checkpoints and figures are saved


In [ ]:
# mount google drive to keep outputs
from google.colab import drive
drive.mount('/gdrive')


In [ ]:
# install needed libraries (torch + medmnist + shap)
# keep versions explicit so results are repeatable
!pip install --quiet torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2
!pip install --quiet medmnist==2.2.2 shap==0.44.1 scikit-learn==1.3.2 matplotlib==3.8.2 seaborn==0.13.1


In [ ]:
# imports
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, models
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
import medmnist
from medmnist import PneumoniaMNIST
import shap

# make runs repeatable
random.seed(7)
np.random.seed(7)
torch.manual_seed(7)
torch.cuda.manual_seed_all(7)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

# folder to save checkpoints and plots inside drive
output_dir = '/gdrive/MyDrive/pneumonia_mnist_resnet18'
os.makedirs(output_dir, exist_ok=True)


## quick gpu check

make sure colab runtime is on gpu. run this cell and you should see cuda:0.


In [ ]:
# check if we have a gpu
print('device:', device)
if device.type != 'cuda':
    print('warning: please switch runtime to gpu for faster training')


## data

pneumonia mnist is already preprocessed. we just load it with medmnist and add a simple normalization.


In [ ]:
# transforms keep channel as single grayscale image
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

train_ds = PneumoniaMNIST(split='train', download=True, transform=img_transform)
val_ds = PneumoniaMNIST(split='val', download=True, transform=img_transform)
test_ds = PneumoniaMNIST(split='test', download=True, transform=img_transform)

batch_size = 128
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=2)

num_classes = len(set(train_ds.labels.reshape(-1).tolist()))
print('classes:', num_classes)


## quick look at a few images

see how the 28x28 grayscale images look. labels: 0 = normal, 1 = pneumonia.


In [ ]:
# show a small batch to understand the data
fig, axes = plt.subplots(2, 5, figsize=(8, 3))
sample_images, sample_labels = next(iter(train_loader))
for ax, img, label in zip(axes.flat, sample_images[:10], sample_labels[:10]):
    ax.imshow(img.squeeze(), cmap='gray')
    ax.set_title(f'label: {label.item()}')
    ax.axis('off')
plt.tight_layout()
plt.show()


## model helper

we use torchvision resnet18 with imagenet weights. first conv is adjusted for 1 channel images. transfer learning happens by choosing which blocks to freeze.


In [ ]:
def build_resnet18(freeze_parts=None):
    # load imagenet pretrained weights
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    # change first conv to accept 1 channel
    base_conv = model.conv1
    model.conv1 = nn.Conv2d(1, base_conv.out_channels, kernel_size=base_conv.kernel_size,
                            stride=base_conv.stride, padding=base_conv.padding, bias=False)
    # average weights of original channels to init
    with torch.no_grad():
        model.conv1.weight = nn.Parameter(base_conv.weight.mean(dim=1, keepdim=True))

    # swap final layer to binary head
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    # freeze selected parts
    if freeze_parts:
        for name, param in model.named_parameters():
            for block_name in freeze_parts:
                if name.startswith(block_name):
                    param.requires_grad = False
                    break
    return model

# examples of freeze configurations
freeze_configs = [
    {'label': 'freeze_to_layer3', 'freeze_parts': ['conv1', 'bn1', 'layer1', 'layer2', 'layer3']},
    {'label': 'freeze_to_layer2', 'freeze_parts': ['conv1', 'bn1', 'layer1', 'layer2']},
    {'label': 'full_finetune', 'freeze_parts': []},
]


## training utils

simple training + validation loops that track best checkpoint per config.


In [ ]:
def run_epoch(model, loader, criterion, optimizer=None):
    model.train() if optimizer else model.eval()
    running_loss = 0.0
    preds, targets = [], []

    for images, labels in loader:
        images = images.to(device)
        labels = labels.squeeze().long().to(device)

        if optimizer:
            optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * images.size(0)

        if optimizer:
            loss.backward()
            optimizer.step()

        pred_labels = torch.argmax(outputs, dim=1)
        preds.extend(pred_labels.detach().cpu().numpy())
        targets.extend(labels.detach().cpu().numpy())

    avg_loss = running_loss / len(loader.dataset)
    acc = accuracy_score(targets, preds)
    return avg_loss, acc


def train_one_setting(label, freeze_parts, epochs=8, lr=1e-3):
    print(f'
>>> training config: {label}')
    model = build_resnet18(freeze_parts).to(device)

    # only update trainable params
    optim_params = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.Adam(optim_params, lr=lr, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()

    best_val_acc = 0.0
    best_path = os.path.join(output_dir, f'{label}_best.pt')

    for epoch in range(1, epochs + 1):
        train_loss, train_acc = run_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_acc = run_epoch(model, val_loader, criterion)
        print(f'epoch {epoch}: train_loss {train_loss:.4f} acc {train_acc:.4f} | val_loss {val_loss:.4f} acc {val_acc:.4f}')

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_path)
            print('saved new best model')
    return best_path, best_val_acc


## run transfer learning experiments

train with different freeze depths to see what works best. adjust epochs upward if the gpu session is stable.


In [ ]:
results = []
for cfg in freeze_configs:
    ckpt, val_acc = train_one_setting(cfg['label'], cfg['freeze_parts'], epochs=10, lr=1e-3)
    results.append({'label': cfg['label'], 'val_acc': val_acc, 'ckpt': ckpt})

# pick best config
results = sorted(results, key=lambda x: x['val_acc'], reverse=True)
best = results[0]
print('
best config:', best)


## evaluation on test set

load the best checkpoint, evaluate, and draw confusion matrix + roc curve.


In [ ]:
# load best model
best_model = build_resnet18()
best_model.load_state_dict(torch.load(best['ckpt'], map_location=device))
best_model.to(device)
best_model.eval()

# collect outputs
all_logits, all_targets = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.squeeze().long().to(device)
        logits = best_model(images)
        all_logits.append(logits.cpu())
        all_targets.append(labels.cpu())

logits = torch.cat(all_logits)
targets = torch.cat(all_targets)
probs = torch.softmax(logits, dim=1)[:, 1]
pred_labels = torch.argmax(logits, dim=1)

acc = accuracy_score(targets, pred_labels)
cm = confusion_matrix(targets, pred_labels)

fpr, tpr, _ = roc_curve(targets, probs)
roc_auc = roc_auc_score(targets, probs)

print('test accuracy:', acc)
print('roc auc:', roc_auc)

# plot confusion matrix
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('predicted')
plt.ylabel('true')
plt.title('confusion matrix')
cm_path = os.path.join(output_dir, 'confusion_matrix.png')
plt.savefig(cm_path, dpi=150, bbox_inches='tight')
plt.show()

# plot roc curve
plt.figure(figsize=(5,4))
plt.plot(fpr, tpr, label=f'auc={roc_auc:.3f}')
plt.plot([0,1],[0,1],'k--')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.title('roc curve')
plt.legend()
roc_path = os.path.join(output_dir, 'roc_curve.png')
plt.savefig(roc_path, dpi=150, bbox_inches='tight')
plt.show()


## shap gradient explainer

use a small background batch for the gradient explainer. we look at one correct and one incorrect prediction (from different classes) to see what pixels drive the decision.


this cell grabs one correct normal case and one incorrect pneumonia case so we see both sides. it also plots a small global mean |shap| map across 64 test images.


In [ ]:
# pick background samples for the explainer
background_images, _ = next(iter(train_loader))
background_images = background_images[:50].to(device)

# prepare explainer
grad_explainer = shap.GradientExplainer(best_model, background_images)

# find one correct normal (label 0) and one incorrect pneumonia (label 1 predicted as 0 or 1)
best_model.eval()
correct_normal = None
incorrect_pneumonia = None

for images, labels in test_loader:
    images = images.to(device)
    labels = labels.squeeze().long().to(device)
    with torch.no_grad():
        outputs = best_model(images)
        preds = torch.argmax(outputs, dim=1)
    for img, label, pred in zip(images, labels, preds):
        if correct_normal is None and label.item() == 0 and pred.item() == 0:
            correct_normal = (img.unsqueeze(0), label.item(), pred.item())
        if incorrect_pneumonia is None and label.item() == 1 and pred.item() != label.item():
            incorrect_pneumonia = (img.unsqueeze(0), label.item(), pred.item())
        if correct_normal and incorrect_pneumonia:
            break
    if correct_normal and incorrect_pneumonia:
        break

examples = {'correct_normal': correct_normal, 'incorrect_pneumonia': incorrect_pneumonia}

for tag, sample in examples.items():
    if sample is None:
        print(f'skipping {tag} because it was not found')
        continue
    img, true_label, pred_label = sample
    shap_values = grad_explainer.shap_values(img)
    shap.image_plot(shap_values, img.cpu().numpy(), show=False)
    plt.title(f'{tag} true={true_label} pred={pred_label}')
    fig_path = os.path.join(output_dir, f'{tag}_shap.png')
    plt.savefig(fig_path, dpi=150, bbox_inches='tight')
    plt.show()

# simple global view: average absolute shap on 64 test images
global_images, global_labels = next(iter(test_loader))
global_images = global_images[:64].to(device)
global_shap = grad_explainer.shap_values(global_images)
# convert list -> array so we can average over classes and samples
global_shap_arr = np.array(global_shap)
mean_abs = np.mean(np.abs(global_shap_arr), axis=(0,1))
mean_abs = np.squeeze(mean_abs)
plt.figure(figsize=(4,4))
plt.imshow(mean_abs, cmap='bwr')
plt.title('global mean |shap| heatmap')
plt.axis('off')
global_path = os.path.join(output_dir, 'global_shap_mean.png')
plt.savefig(global_path, dpi=150, bbox_inches='tight')
plt.show()


## notes for the report

* keep an eye on whether freezing more layers hurts validation accuracy. usually unfreezing layer4 helps.
* confusion matrix shows where false positives/negatives happen.
* shap maps highlight lung regions that push decisions; noisy edges suggest overfitting.
* store your favorite checkpoint and figures in drive for later write-up.
